In [1]:
# Environment setup
import os
import json
import json5
from uuid import UUID
import nest_asyncio
from dotenv import load_dotenv

nest_asyncio.apply()
load_dotenv(".env")  # Load your OpenAI API key from .env file

from pathlib import Path
import pathlib

from lmi import CommonLLMNames
from paperqa.settings import Settings
from paperqa.agents.main import agent_query
from paperqa.agents.search import get_directory_index

from aviary.envs.litqa.env import GradablePaperQAEnvironment
from ldp.agent import SimpleAgent
from ldp.alg import Evaluator, EvaluatorConfig, MeanMetricsCallback
from aviary.env import TaskDataset
from aviary.envs.litqa.task import TASK_DATASET_NAME

/Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/pybtex/plugin/__init__.py:26: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In terminal:

chmod +x Meta-Llama-3.1-8B-Instruct.Q4_K_M.llamafile

./Meta-Llama-3.1-8B-Instruct.Q4_K_M.llamafile -cb -np 4 -a my-llama-model

In [2]:
llm_llama = "my-llama-model"
llm_openai = CommonLLMNames.OPENAI_TEST.value

from paperqa.prompts import (
    CONTEXT_INNER_PROMPT,
    CONTEXT_OUTER_PROMPT,
    citation_prompt,
    default_system_prompt,
    env_reset_prompt,
    env_system_prompt,
    qa_prompt,
    select_paper_prompt,
    structured_citation_prompt,
    summary_json_prompt,
    summary_json_system_prompt,
    summary_prompt,
)
from paperqa.settings import (
    AgentSettings,
    AnswerSettings,
    IndexSettings,
    ParsingSettings,
    PromptSettings,
    Settings,
)

settings = Settings(
    llm=llm_llama,
    llm_config = {
        "model_list": [
            {
                "model_name": llm_llama,
                "litellm_provider": "openai",
                "litellm_params": {
                    "model": llm_llama,
                    "api_base": "http://localhost:8081/v1",
                    "api_key": "sk-no-key-required",
                    "temperature": 0.1,
                    "frequency_penalty": 1.5,
                    "max_tokens": 512,
                },
            }
        ],
        "rate_limit": {
            llm_llama: "30000 per 1 minute",
        },
    },
    summary_llm=llm_llama,
    summary_llm_config={
        "rate_limit": {
            llm_llama: "30000 per 1 minute",
        },
    },
    embedding="text-embedding-3-small",
    embedding_config={
        "model_list": [
            {
                "model_name": llm_openai,
                "litellm_params": {
                    "model": llm_openai,
                    "temperature": 0.1,
                    "max_tokens": 4096,
                },
            }
        ],
        "rate_limit": {
            llm_openai: "30000 per 1 minute",
        },
    },
    temperature=0.1,
    batch_size=1,
    verbosity=1,
    paper_directory=pathlib.Path.cwd().joinpath("litqa_pdfs"),
    index_directory=pathlib.Path.cwd().joinpath("litqa_pdfs/index"),
    answer=AnswerSettings(
        evidence_k=5,
        evidence_detailed_citations=True,
        evidence_retrieval=True,
        evidence_summary_length="about 100 words",
        evidence_skip_summary=False,
        answer_max_sources=5,
        max_answer_attempts=None,
        answer_length="about 200 words, but can be longer",
        max_concurrent_requests=10,
    ),
    parsing=ParsingSettings(
        chunk_size=5000,
        overlap=250,
        citation_prompt=citation_prompt,
        structured_citation_prompt=structured_citation_prompt,
    ),
    prompts=PromptSettings(
        summary=summary_prompt,
        qa=qa_prompt,
        select=select_paper_prompt,
        pre=None,
        post=None,
        system=default_system_prompt,
        use_json=True,
        summary_json=summary_json_prompt,
        summary_json_system=summary_json_system_prompt,
        context_outer=CONTEXT_OUTER_PROMPT,
        context_inner=CONTEXT_INNER_PROMPT,
    ),
    agent=AgentSettings(
        agent_llm=llm_llama,
        agent_llm_config={
            "model_list": [
                {
                    "model_name": llm_llama,
                    "litellm_params": {
                        "model": llm_llama,
                    },
                }
            ],
            "rate_limit": {
                llm_llama: "30000 per 1 minute",
            },
        },
        agent_prompt=env_reset_prompt,
        agent_system_prompt=env_system_prompt,
        search_count=8,
        index=IndexSettings(
            paper_directory=pathlib.Path.cwd().joinpath("litqa_pdfs"),
            index_directory=pathlib.Path.cwd().joinpath("litqa_pdfs/index"),
        ),
    ),
)

embedding="text-embedding-ada-002" ?

In [3]:
# Load the LitQA v2 dataset (eval split)
dataset = TaskDataset.from_name(TASK_DATASET_NAME, settings=settings)

records = []
for i in range(len(dataset)):
    env = dataset.get_new_env_by_idx(i)
    q = env._query

    record = {
        "id": str(q.question_id) if isinstance(q.question_id, UUID) else q.question_id,
        "question": q.question,
        "options": q.options,
        "correct_answer": q.ideal_answer,
        "sources": env.sources,
    }
    records.append(record)

with open("data/litqa_eval_repro_llama.json", "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2, ensure_ascii=False)

print(f"Saved {len(records)} questions to litqa_eval_repro_llama.json")

/Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Saved 40 questions to litqa_eval_repro_llama.json


In [ ]:
from paperqa import ask

response = ask("What is paperqa2?", settings=settings)

PaperQA version: 5.21.1.dev17+g2cf0d6c


[18:33:27] Beginning agent 'ToolSelector' run with question 'What is paperqa2?' and full settings {'llm':          
           'my-llama-model', 'llm_config': {'model_list': [{'model_name': 'my-llama-model', 'litellm_provider':    
           'openai', 'litellm_params': {'model': 'my-llama-model', 'api_base': 'http://localhost:8081/v1',         
           'api_key': 'sk-no-key-required', 'temperature': 0.1, 'frequency_penalty': 1.5, 'max_tokens': 512}}],    
           'rate_limit': {'my-llama-model': '30000 per 1 minute'}}, 'summary_llm': 'my-llama-model',               
           'summary_llm_config': {'rate_limit': {'my-llama-model': '30000 per 1 minute'}}, 'embedding':            
           'text-embedding-3-small', 'embedding_config': {'model_list': [{'model_name': 'gpt-4o-mini-2024-07-18',  
           'litellm_params': {'model': 'gpt-4o-mini-2024-07-18', 'temperature': 0.1, 'max_tokens': 4096}}],        
           'rate_limit': {'gpt-4o-mini-2024-07-18': '30000 per 1 minute'}}, 'temperature': 0.1, 'batch_size': 1,   
           'texts_index_mmr_lambda': 1.0, 'verbosity': 1, 'answer': {'evidence_k': 5,                              
           'evidence_detailed_citations': True, 'evidence_retrieval': True, 'evidence_summary_length': 'about 100  
           words', 'evidence_skip_summary': False, 'answer_max_sources': 5, 'max_answer_attempts': None,           
           'answer_length': 'about 200 words, but can be longer', 'max_concurrent_requests': 10,                   
           'answer_filter_extra_background': False, 'get_evidence_if_no_contexts': True}, 'parsing': {'chunk_size':
           5000, 'page_size_limit': 1280000, 'pdfs_use_block_parsing': False, 'use_doc_details': True, 'overlap':  
           250, 'citation_prompt': 'Provide the citation for the following text in MLA Format. Do not write an     
           introductory sentence. If reporting date accessed, the current year is 2025\n\n{text}\n\nCitation:',    
           'structured_citation_prompt': "Extract the title, authors, and doi as a JSON from this MLA citation. If 
           any field can not be found, return it as null. Use title, authors, and doi as keys, author's value      
           should be a list of authors. {citation}\n\nCitation JSON:", 'disable_doc_valid_check': False,           
           'defer_embedding': False, 'chunking_algorithm': <ChunkingOptions.SIMPLE_OVERLAP: 'simple_overlap'>,     
           'doc_filters': None, 'use_human_readable_clinical_trials': False}, 'prompts': {'summary': 'Summarize the
           excerpt below to help answer a question.\n\nExcerpt from                                                
           {citation}\n\n----\n\n{text}\n\n----\n\nQuestion: {question}\n\nDo not directly answer the question,    
           instead summarize to give evidence to help answer the question. Stay detailed; report specific numbers, 
           equations, or direct quotes (marked with quotation marks). Reply "Not applicable" if the excerpt is     
           irrelevant. At the end of your response, provide an integer score from 1-10 on a newline indicating     
           relevance to question. Do not explain your score.\n\nRelevant Information Summary ({summary_length}):', 
           'qa': 'Answer the question below with the context.\n\nContext (with relevance                           
           scores):\n\n{context}\n\n----\n\nQuestion: {question}\n\nWrite an answer based on the context. If the   
           context provides insufficient information reply "I cannot answer." For each part of your answer,        
           indicate which sources most support it via citation keys at the end of sentences, like                  
           {example_citation}. Only cite from the context above and only use the citation keys from the context. ##
           Valid citation examples: \n- Example2024Example pages 3-4 \n- Example2024 pages 3-4 \n- Example2024     
           pages 3-4, Example2024 pages 5-6 \n## Invalid

           New file to index: 10.1038%2Fs41586-023-05906-y.pdf...


Provider List: https://docs.litellm.ai/docs/providers



[18:33:28] Error parsing 10.1038%2Fs41586-023-05906-y.pdf, skipping index for this file.                           
           ╭───────────────────────────────── Traceback (most recent call last) ──────────────────────────────────╮
           │ /Users/jiali03/Desktop/xc392/paper-qa/paperqa/agents/search.py:530 in process_file                   │
           │                                                                                                      │
           │   527 │   │   │                                                                                      │
           │   528 │   │   │   tmp_docs = Docs()                                                                  │
           │   529 │   │   │   try:                                                                               │
           │ ❱ 530 │   │   │   │   await tmp_docs.aadd(                                                           │
           │   531 │   │   │   │   │   path=abs_file_path,                                                        │
           │   532 │   │   │   │   │   fields=["title", "author", "journal", "year"],                             │
           │   533 │   │   │   │   │   settings=settings,                                                         │
           │                                                                                                      │
           │ /Users/jiali03/Desktop/xc392/paper-qa/paperqa/docs.py:305 in aadd                                    │
           │                                                                                                      │
           │   302 │   │   │   )                                                                                  │
           │   303 │   │   │   if not texts:                                                                      │
           │   304 │   │   │   │   raise ValueError(f"Could not read document {path}. Is it empty?")              │
           │ ❱ 305 │   │   │   result = await llm_model.call_single(                                              │
           │   306 │   │   │   │   messages=[                                                                     │
           │   307 │   │   │   │   │   Message(                                                                   │
           │   308 │   │   │   │   │   │   content=parse_config.citation_prompt.format(text=texts[0].text)        │
           │                                                                                                      │
           │ /Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/lmi/llms.py:386 in          │
           │ call_single                                                                                          │
           │                                                                                                      │
           │   383 │   │   tool_choice: Tool | str | None = TOOL_CHOICE_REQUIRED,                                 │
           │   384 │   │   **kwargs,                                                                              │
           │   385 │   ) -> LLMResult:                                                                            │
           │ ❱ 386 │   │   results = await self.call(                                                             │
           │   387 │   │   │   messages, callbacks, name, output_type, tools, tool_choice, n=1, **kwargs          │
           │   388 │   │   )                                                                                      │
           │   389 │   │   if not results:                                                                        │
           │                                                                                                      │
           │ /Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/lmi/llms.py:266 in call     │
           │                                            

...


Provider List: https://docs.litellm.ai/docs/providers



           Error parsing 10.1016%2Fj.chom.2024.03.010.pdf, skipping index for this file.                           
           ╭───────────────────────────────── Traceback (most recent call last) ──────────────────────────────────╮
           │ /Users/jiali03/Desktop/xc392/paper-qa/paperqa/agents/search.py:530 in process_file                   │
           │                                                                                                      │
           │   527 │   │   │                                                                                      │
           │   528 │   │   │   tmp_docs = Docs()                                                                  │
           │   529 │   │   │   try:                                                                               │
           │ ❱ 530 │   │   │   │   await tmp_docs.aadd(                                                           │
           │   531 │   │   │   │   │   path=abs_file_path,                                                        │
           │   532 │   │   │   │   │   fields=["title", "author", "journal", "year"],                             │
           │   533 │   │   │   │   │   settings=settings,                                                         │
           │                                                                                                      │
           │ /Users/jiali03/Desktop/xc392/paper-qa/paperqa/docs.py:305 in aadd                                    │
           │                                                                                                      │
           │   302 │   │   │   )                                                                                  │
           │   303 │   │   │   if not texts:                                                                      │
           │   304 │   │   │   │   raise ValueError(f"Could not read document {path}. Is it empty?")              │
           │ ❱ 305 │   │   │   result = await llm_model.call_single(                                              │
           │   306 │   │   │   │   messages=[                                                                     │
           │   307 │   │   │   │   │   Message(                                                                   │
           │   308 │   │   │   │   │   │   content=parse_config.citation_prompt.format(text=texts[0].text)        │
           │                                                                                                      │
           │ /Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/lmi/llms.py:386 in          │
           │ call_single                                                                                          │
           │                                                                                                      │
           │   383 │   │   tool_choice: Tool | str | None = TOOL_CHOICE_REQUIRED,                                 │
           │   384 │   │   **kwargs,                                                                              │
           │   385 │   ) -> LLMResult:                                                                            │
           │ ❱ 386 │   │   results = await self.call(                                                             │
           │   387 │   │   │   messages, callbacks, name, output_type, tools, tool_choice, n=1, **kwargs          │
           │   388 │   │   )                                                                                      │
           │   389 │   │   if not results:                                                                        │
           │                                                                                                      │
           │ /Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/lmi/llms.py:266 in call     │
           │                                            

           New file to index: 10.1101%2F2024.02.20.581217.pdf...


Provider List: https://docs.litellm.ai/docs/providers



[18:33:31] Error parsing 10.1101%2F2024.02.20.581217.pdf, skipping index for this file.                            
           ╭───────────────────────────────── Traceback (most recent call last) ──────────────────────────────────╮
           │ /Users/jiali03/Desktop/xc392/paper-qa/paperqa/agents/search.py:530 in process_file                   │
           │                                                                                                      │
           │   527 │   │   │                                                                                      │
           │   528 │   │   │   tmp_docs = Docs()                                                                  │
           │   529 │   │   │   try:                                                                               │
           │ ❱ 530 │   │   │   │   await tmp_docs.aadd(                                                           │
           │   531 │   │   │   │   │   path=abs_file_path,                                                        │
           │   532 │   │   │   │   │   fields=["title", "author", "journal", "year"],                             │
           │   533 │   │   │   │   │   settings=settings,                                                         │
           │                                                                                                      │
           │ /Users/jiali03/Desktop/xc392/paper-qa/paperqa/docs.py:305 in aadd                                    │
           │                                                                                                      │
           │   302 │   │   │   )                                                                                  │
           │   303 │   │   │   if not texts:                                                                      │
           │   304 │   │   │   │   raise ValueError(f"Could not read document {path}. Is it empty?")              │
           │ ❱ 305 │   │   │   result = await llm_model.call_single(                                              │
           │   306 │   │   │   │   messages=[                                                                     │
           │   307 │   │   │   │   │   Message(                                                                   │
           │   308 │   │   │   │   │   │   content=parse_config.citation_prompt.format(text=texts[0].text)        │
           │                                                                                                      │
           │ /Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/lmi/llms.py:386 in          │
           │ call_single                                                                                          │
           │                                                                                                      │
           │   383 │   │   tool_choice: Tool | str | None = TOOL_CHOICE_REQUIRED,                                 │
           │   384 │   │   **kwargs,                                                                              │
           │   385 │   ) -> LLMResult:                                                                            │
           │ ❱ 386 │   │   results = await self.call(                                                             │
           │   387 │   │   │   messages, callbacks, name, output_type, tools, tool_choice, n=1, **kwargs          │
           │   388 │   │   )                                                                                      │
           │   389 │   │   if not results:                                                                        │
           │                                                                                                      │
           │ /Users/jiali03/Desktop/xc392/paper-qa/.venv/lib/python3.13/site-packages/lmi/llms.py:266 in call     │
           │                                            

In [ ]:
async def evaluate():
    metrics_callback = MeanMetricsCallback(eval_dataset=dataset)
    evaluator = Evaluator(
        config=EvaluatorConfig(batch_size=3),
        agent=SimpleAgent(),
        dataset=dataset,
        callbacks=[metrics_callback],
    )
    await evaluator.evaluate()

    # Save result to JSON file
    with open("results/gpt_eval_results_llama.json", "w", encoding="utf-8") as f:
        json.dump(metrics_callback.eval_means, f, indent=2, ensure_ascii=False)

await evaluate()

## Use Ollama

In terminal:

curl -fsSL https://ollama.com/install.sh | sh
ollama pull llama3:instruct
ollama pull mxbai/embed-large

from paperqa import Settings, ask

llm_config = {
    "model_list": [
        {
            "model_name": "ollama/llama3.2",
            "litellm_params": {
                "model": "ollama/llama3.2",
                "api_base": "http://localhost:11434",
            },
        }
    ]
}

settings = Settings(
    llm="ollama/llama3.2",
    llm_config=llm_config,
    summary_llm="ollama/llama3.2",
    summary_llm_config=llm_config,
    embedding="ollama/mxbai-embed-large",
)

response = ask("What is PaperQA2?", settings=settings)
print(response.answer)